In [ ]:
# Import some basic libraries and functions for this tutorial.
import json
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
from PIL import Image
import os
import shutil
import warnings

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    Parser,
    RandomGenerator,
    RigidTransform,
    Role,
    RollPitchYaw,
    Simulator,
    UniformlyRandomRotationMatrix,
    RotationMatrix
)

# from manipulation import running_as_notebook
# from manipulation.scenarios import AddRgbdSensor
# from manipulation.utils import colorize_labels

from pydrake.common import FindResourceOrThrow, temp_directory
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)

import sys

from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import Parser
from glob import glob

import multiprocessing as mp
from tqdm.autonotebook import tqdm

import yaml

import cv2

import trimesh

from pydrake.all import (
    AbstractValue, Adder, AddMultibodyPlantSceneGraph, BallRpyJoint, BaseField,
    Box, CameraInfo, ClippingRange, CoulombFriction, Cylinder, Demultiplexer,
    DepthImageToPointCloud, DepthRange, DepthRenderCamera, DiagramBuilder,
    FindResourceOrThrow, GeometryInstance, InverseDynamicsController,
    LeafSystem, LoadModelDirectivesFromString,
    MakeMultibodyStateToWsgStateSystem, MakePhongIllustrationProperties,
    MakeRenderEngineVtk, ModelInstanceIndex, MultibodyPlant, Parser,
    PassThrough, PrismaticJoint, ProcessModelDirectives, RenderCameraCore,
    RenderEngineVtkParams, RevoluteJoint, Rgba, RgbdSensor, RigidTransform,
    RollPitchYaw, RotationMatrix, SchunkWsgPositionController, SpatialInertia,
    Sphere, StateInterpolatorWithDiscreteDerivative, UnitInertia)

from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters)

In [ ]:
def AddRgbdSensor(builder,
                  scene_graph,
                  X_PC,
                  depth_camera=None,
                  renderer=None,
                  parent_frame_id=None):
    """
    Adds a RgbdSensor to to the scene_graph at (fixed) pose X_PC relative to
    the parent_frame.  If depth_camera is None, then a default camera info will
    be used.  If renderer is None, then we will assume the name 'my_renderer',
    and create a VTK renderer if a renderer of that name doesn't exist.  If
    parent_frame is None, then the world frame is used.
    """
    if sys.platform == "linux" and os.getenv("DISPLAY") is None:
        from pyvirtualdisplay import Display
        virtual_display = Display(visible=0, size=(1400, 900))
        virtual_display.start()

    if not renderer:
        renderer = "my_renderer"

    if not parent_frame_id:
        parent_frame_id = scene_graph.world_frame_id()

    if not scene_graph.HasRenderer(renderer):
        scene_graph.AddRenderer(renderer,
                                MakeRenderEngineVtk(RenderEngineVtkParams()))

    if not depth_camera:
        depth_camera = DepthRenderCamera(
            RenderCameraCore(
                renderer, CameraInfo(width=640, height=480, fov_y=np.pi / 4.0),
                ClippingRange(near=0.1, far=10.0), RigidTransform()),
            DepthRange(0.1, 10.0))

    rgbd = builder.AddSystem(
        RgbdSensor(parent_id=parent_frame_id,
                   X_PB=X_PC,
                   depth_camera=depth_camera,
                   show_window=False))

    builder.Connect(scene_graph.get_query_output_port(),
                    rgbd.query_object_input_port())

    return rgbd

In [ ]:
objects = ['055_baseball.sdf',
 '056_tennis_ball.sdf',
 '072-a_toy_airplane.sdf',
 '019_pitcher_base.sdf',
 '040_large_marker.sdf',
 '021_bleach_cleanser.sdf',
 '077_rubiks_cube.sdf',
 '048_hammer.sdf',
 '008_pudding_box.sdf',
 '053_mini_soccer_ball.sdf',
 '011_banana.sdf',
 '006_mustard_bottle.sdf',
 '013_apple.sdf',
 '029_plate.sdf',
 '035_power_drill.sdf',
 '043_phillips_screwdriver.sdf',
 '032_knife.sdf',
 '042_adjustable_wrench.sdf']
noise_objects = ['050_medium_clamp.sdf',
'002_master_chef_can.sdf',
'009_gelatin_box.sdf',
'015_peach.sdf',
'054_softball.sdf',
'003_cracker_box.sdf',
'012_strawberry.sdf',
'026_sponge.sdf',
'018_plum.sdf',
'004_sugar_box.sdf',
'014_lemon.sdf',
'071_nine_hole_peg_test.sdf',
'057_racquetball.sdf',
'030_fork.sdf',
'007_tuna_fish_can.sdf',
'024_bowl.sdf',
'062_dice.sdf',
'037_scissors.sdf',
'058_golf_ball.sdf',
'052_extra_large_clamp.sdf',
'044_flat_screwdriver.sdf',
'061_foam_brick.sdf',
'033_spatula.sdf',
'016_pear.sdf',
'025_mug.sdf',
'010_potted_meat_can.sdf',
'017_orange.sdf',
'036_wood_block.sdf'
]

In [ ]:
obj_dict = {}
id_dict = {}
mask_dict = {}
mask_vals = np.linspace(0,255,len(objects)+1,dtype=np.int32)[1:]
for i,this_object in enumerate(objects):
    obj_dict[i] = this_object.replace('.sdf','')
    id_dict[this_object.replace('.sdf','')] = i+1
    mask_dict[this_object.replace('.sdf','')] = mask_vals[i]
print(obj_dict)
print(id_dict)
print(mask_dict)

In [ ]:
os.system('rm -r ./dataset/')
os.system('mkdir ./dataset/' )
os.system('mkdir ./dataset/rgb/')
os.system('mkdir ./dataset/merged_masks/')
os.system('mkdir ./dataset/masks/')
os.system('mkdir ./dataset/depth/')
for n in mask_dict.keys():
    os.system('mkdir ./dataset/masks/' + n + '/')

In [ ]:
def run_imap_multiprocessing(func, argument_list, show_prog = True):
    pool = mp.Pool(processes=mp.cpu_count())
    
    if show_prog:            
        result_list_tqdm = []
        for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list),position=0, leave=True):
            result_list_tqdm.append(result)
    else:
        result_list_tqdm = []
        for result in pool.imap(func=func, iterable=argument_list):
            result_list_tqdm.append(result)

    return result_list_tqdm

In [ ]:
def generate_sample(n):
    np.random.seed(int.from_bytes(os.urandom(4), byteorder='little'))
    rng = np.random.default_rng()  # this is for python
    generator = RandomGenerator(rng.integers(1000))  # for c++
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0001)
    parser = Parser(plant)
    table_sdf = "./table/table.sdf"
    parser.AddModelFromFile(table_sdf)
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table_body_link"))

    parser.AddModelFromFile("ground.sdf")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("ground_link"))

    parser.AddModelFromFile("table_top.sdf")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table_top_link"))


    inspector = scene_graph.model_inspector()

    instance_id_to_class_name = dict()
    
    n_target = np.random.randint(low=5,high=20)
    rand_i = np.random.choice(len(objects),size=n_target)
    n_noise = np.random.randint(low=5,high=10)
    rand_i_n = np.random.choice(len(noise_objects),size=n_noise)
    
    for object_num in range(n_target):

        this_object = objects[rand_i[object_num]]
        class_name = this_object.split('.')[0]
        sdf = "./ycb/sdf/"+ this_object
        instance = parser.AddModelFromFile(sdf, f"object{class_name}_%i"%(object_num))

        frame_id = plant.GetBodyFrameIdOrThrow(
            plant.GetBodyIndices(instance)[0])
        geometry_ids = inspector.GetGeometries(frame_id, Role.kPerception)
        for geom_id in geometry_ids:
            instance_id_to_class_name[int(
                inspector.GetPerceptionProperties(geom_id).GetProperty(
                    "label", "id"))] = class_name

    for object_num in range(n_noise):

        this_object = noise_objects[rand_i_n[object_num]]
        class_name = this_object.split('.')[0]
        sdf = "./ycb/sdf/"+ this_object
        instance = parser.AddModelFromFile(sdf, f"object{class_name}_%i"%(object_num))
            
    plant.Finalize()

    thetas = np.linspace(0,np.pi,20)
    if np.random.uniform()<0.5:
        p1 = np.random.uniform(low=[-0.3,-0.2,0.5],high=[-0.3,0.2,0.5])
    else:
        p1 = np.random.uniform(low=[-0.3,-0.2,0.5],high=[0.3,-0.2,0.5])
    p2 = -p1
    p2[-1] = 0.5

    l = p1-p2
    l = l[0:2]/np.linalg.norm(l[0:2])
    p1[0:2] = p1[0:2] + l * 1.0
    p2[0:2] = p2[0:2] - l * 1.0
    r = np.linalg.norm(p1[0:2]-p2[0:2])/2
    cc = (p1[0:2]+p2[0:2])/2
    r_u = p1[0:2] - cc
    cameras = []
    for i in range(20):
        # pick two points on the table to circle over
        rr = r_u * np.cos(thetas[i])
        zr = r * np.sin(thetas[i])
        p = cc + rr
        p = np.array([p[0],p[1],zr+0.5])
        r_c = np.array([0.,0.,0.4]-p)
        r_c = r_c/np.linalg.norm(r_c)
        c_uv = np.array([0.,0.,1.])
        v = np.cross(c_uv,r_c)
        s = np.linalg.norm(v)
        c = np.dot(c_uv,r_c)
        v_c = np.array([[0,-v[2],v[1]],
                        [v[2],0,-v[0]],
                        [-v[1],v[0],0]])
        R = np.eye(3) + v_c + v_c@v_c * ((1-c)/s**2)

        cameras.append(AddRgbdSensor(
            builder, scene_graph,
            RigidTransform(RotationMatrix(R), p)))
        cameras[-1].set_name("rgbd_sensor_" + str(i+1))
        builder.ExportOutput(cameras[-1].color_image_output_port(), "color_image_" + str(i+1))
        builder.ExportOutput(cameras[-1].label_image_output_port(), "label_image_" + str(i+1))
        builder.ExportOutput(cameras[-1].depth_image_32F_output_port(),"depth_image_" + str(i+1))
        builder.ExportOutput(cameras[-1].body_pose_in_world_output_port(),"camera_pose_" + str(i+1))

    diagram = builder.Build()
    cc = 0
    while True:
        simulator = Simulator(diagram)
        context = simulator.get_mutable_context()
        plant_context = plant.GetMyContextFromRoot(context)

        z = 0.6
        c = 0
        for body_index in plant.GetFloatingBaseBodies():
            tf = RigidTransform(
                UniformlyRandomRotationMatrix(generator),
                np.random.uniform(low=[-0.3,-0.2,0.5 + c * 0.1],high=[0.3,0.2,0.5 + c * 0.1],size=3).tolist())
            # tf = RigidTransform()
            # tf.set_translation([-0.5 + (c%5)*0.2,-0.2 + (c//5) * 0.2, 1.0])
            c += 1
            plant.SetFreeBodyPose(plant_context, plant.get_body(body_index), tf)
            z += 0.1

        try:
            simulator.AdvanceTo(3.0)
            break
        except RuntimeError:
            # I've chosen an aggressive simulation time step which works most
            # of the time, but can fail occasionally.
            pass
        if cc>=100:
            return generate_sample(n)
        cc += 1
    pose_output = []
    for i in range(20):
        img_i = n*20 + i
        color_image = diagram.GetOutputPort("color_image_%i"%(i+1)).Eval(context)
        label_image = diagram.GetOutputPort("label_image_%i"%(i+1)).Eval(context)
        depth_image = diagram.GetOutputPort("depth_image_%i"%(i+1)).Eval(context)
        X_WC = diagram.GetOutputPort("camera_pose_%i"%(i+1)).Eval(context)
        X_CW = X_WC.inverse()
        
        im = Image.fromarray(color_image.data)
        im.save('./dataset/rgb/%i.png'% img_i)
        
        np.save('./dataset/depth/%i.npy'% img_i,depth_image.data)
        
        instance_id_to_pos = dict()

        for body_index in plant.GetFloatingBaseBodies():
            instance_id_to_pos[int(body_index)] = X_CW @ plant.GetFreeBodyPose(plant_context, plant.get_body(body_index))
            
        visible_objects = np.unique(label_image.mutable_data)
        
        poses = []
        merged_mask = np.zeros_like(label_image.mutable_data.squeeze()).astype(np.uint8)
        
        instance_count = {}
        
        for k in mask_dict.keys():
            instance_count[k] = 0
        
        for i,c in enumerate(instance_id_to_class_name):
            if c in visible_objects:
                mask = label_image.mutable_data == c
                pos = np.where(mask)
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                
                if mask.sum() > 9 and xmin<xmax and ymin<ymax:
                    im = Image.fromarray((mask*255).astype(np.uint8).squeeze())
                    im.save('./dataset/masks/'+ instance_id_to_class_name[c] + '/%i.png'% img_i)
                    poses.append([instance_id_to_class_name[c],instance_id_to_pos[c].translation(),instance_id_to_pos[c].rotation().matrix()])
                    mask_val = mask_dict[instance_id_to_class_name[c]] - instance_count[instance_id_to_class_name[c]]
                    instance_count[instance_id_to_class_name[c]] += 1
                    if instance_count[instance_id_to_class_name[c]] > 10:
                        return generate_sample(n)
                    merged_mask += mask.astype(np.uint8).squeeze() * mask_val
        
        im = Image.fromarray(merged_mask)
        im.save('./dataset/merged_masks/%i.png'% img_i)
        
        pose_output.append((img_i,poses))
        
    return pose_output

In [ ]:
pos_out = run_imap_multiprocessing(generate_sample,list(range(1250)))

In [ ]:
poses = []
cam_K = [579.4112549695428, 0.0, 319.5, 0.0, 579.4112549695428, 239.5, 0.0, 0.0, 1.0]
for p in pos_out:
    poses += p

if not 'ground_truth' in locals():
    ground_truth = {}

for p in poses:
    pose_list = []
    for item in p[1]:
        pose_list.append({'cam_R_m2c':item[2].flatten().tolist(),'cam_t_m2c':item[1].flatten().tolist(),'obj_id': id_dict[item[0]],'cam_K': cam_K})
    ground_truth[p[0]] = pose_list
    
with open(r'./dataset/gt.yml', 'w') as file:
    documents = yaml.dump(ground_truth, file,default_flow_style=False)